In [45]:
suppressMessages(library(rwwa))

# Fit linear model to gridded data

## Edit this cell to load the correct data and define the model you want to fit

In [47]:
# use the same filename from the last workbook
fnm_root = "rx1day-era5-260126"

# load covariate data
gmst = read.table("ts-obs/gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2026]

nao <- load_ts("ts-obs/med-storms_nao-djf_era5-stn.dat", col.names = c("year", "nao"))
cov_df <- merge(gmst, nao)

# define factual & counterfactual climates
cov_f <- cov_df[cov_df$year == 2026,c(-1),drop = F]
cov_cf <- rbind("pi" = cov_f - c(1.3,0),
                "naoneutral" = c(cov_f$gmst, 0),
                "pineutral" = c(-1.3, 0))

In [48]:
# specify the statistical model you want to fit (should be the same as for your main event definition)
dist <- "gev"
fittype <- "fixeddisp"
covnm <- c("gmst", "nao")
lower = F            # use T to look at high extremes, F to look at low extremes

## Run this cell with no changes to fit the model at each grid cell

In [49]:
# loop over all flattened data & fit nonstationary model to each - NB no bootstrapping
fl = list.files("flattened", full.names = T)
for(fnm in fl) {
    
    res_fnm <- gsub("flattened", "flattened-res", fnm)
    
    if (!file.exists(res_fnm)) {
        
        # load the .csv and extract the event
        df <- read.csv(fnm, skip = 3, header = F)
        colnames(df)[1] <- "year"
        df$year[1] <- "ev"

        df <- merge(cov_df,df, all.y = T)

        # fit nonstationary model to all columns - specify event value
        models <- sapply(colnames(df)[grepl("V", colnames(df))], function(cnm) {
            df_c <- df[,c("year", c(covnm), cnm)]
            tryCatch({
                fit_ns(dist, fittype, data = df_c[!is.na(df_c$gmst),], covnm = covnm, varnm = cnm, lower = F,
                       ev = df_c[df_c$year == "ev",cnm], ev_year = 2026)
            }, error = function(cond) { return(NULL) })
        }, simplify = F)

        # get all model results & save as .csv to be reconstructed & plotted in python
        fitted_res <- sapply(models, function(mdl) {
            if(!is.null(unlist(mdl)[1])) {
                mdl_ests(mdl, cov_f = cov_f, cov_cf = cov_cf)
            } else {
                rep(NA, 11)
            }})
        write.csv(fitted_res, res_fnm)
    }
}

**Now open the next workbook (using python) to reconstruct the fitted results into a map**